# 02456 Deep Learning - Acne Vulgaris vs Acne Rosacea


### Model: Retrain last layer of Inception V3 - With Spatial Transformer

#### Load the weights of the trained Inception V3 model, the weights of the trained localization net and evaluate some test images found using google image search (As provided training data is classified)

This notebook is used to show that the network works, by classifying an input image containing lesions of either Acne Vulgaris or Acne Rosacea. It is possible to use your own images by changing or adding the vector of image URLS. The network outputs a transformed image, and a probability of class A or B. 

Authors:
* s134859
* sxxxxxx
* sxxxxxx

References: